<h1>Support functions and classes</h1>

In [3]:
import numpy as np
import math
import random
import scipy.stats as stat
from timeit import default_timer as timer
import scipy.misc
import scipy
import operator
import gensim
import logging
import sys
from scipy import spatial
from bs4 import BeautifulSoup
import lxml
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import collections
from enum import Enum
import itertools
import glob
import cython
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['legend.loc'] = 'best'
import profile

In [4]:
### support classes and functions

# for floats comparison
def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
    return abs(a - b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)


def getDocuments(filename, processed=False,max_docs=None):
    print('reading documents')
    if (processed == False):
        vocab = Vocabulary()
        pr = Preprocessor(vocab)
    infile = open(filename)
    raw = infile.read()
    infile.close()
    res = {}
    soup = BeautifulSoup(raw, "lxml")
    for i,doc in enumerate(soup.findAll("doc")):
        if(max_docs!=None and max_docs==i): break

        headline = doc.find('headline')
        maintext = doc.find('text')
        headline = headline = headline.text if headline else ""
        maintext = maintext = maintext.text if maintext else ""

        if (processed == False):
            headline = pr.prep(headline)
            maintext = pr.prep(maintext)
        else:
            headline=headline.split(" ")
            maintext=maintext.split(" ")

        # for empty sections
        if(len(headline)==1 and headline[0]==""):
            headline=[]

        if(len(maintext)==1 and maintext[0]==""):
            maintext=[]

        docno = doc.find('docno').text

        # making sure that we have ints
        headline = [int(x) for x in headline]
        maintext = [int(x) for x in maintext]

        res[docno.strip()] = {'headline': headline, 'maintext': maintext, 'fulltext': headline +maintext}

    print('finished reading documents')
    if (processed == False):
        vocab.add("UNKNOWN")
        return (res, vocab)
    else:
        return (res)


def getQueries(filename, vocab):
    queries = {}
    pr = Preprocessor(vocab)
    with open(filename) as f:
        for line in f:
            parts = line.split(" ")
            # parts.pop(0) # remove id
            queries[parts[0]] = pr.prep(' '.join(parts), 1)
    return queries


def getDocQuerRel(filename):
    qdr = {}  # maps: qid->{did->rel}
    with open(filename) as f:
        for line in f:
            parts = line.split(" ")
            if (parts[0] not in qdr):
                qdr[parts[0]] = {}
            qdr[parts[0]][parts[2]] = int(parts[3])
    return qdr


# writes preprocessed documents into a file
def writeDocuments(docs, output_file):
    with open(output_file, 'w') as f:
        for did in docs:
            headline = ' '.join(str(x) for x in docs[did]['headline'])
            maintext = ' '.join(str(x) for x in docs[did]['maintext'])
            doc = "<doc>" \
                  "<docno>" + did + "</docno>" \
                                    "<headline>" + headline + "</headline>" \
                                                              "<text>" + maintext + "</text>" \
                                                                                    "</doc>"
            f.write(doc + "\n")
    print("Documents written to " + output_file)


def writeVocabulary(vocab, output_file, sep=' '):
    with open(output_file, 'w') as f:
        f.write("\n".join([sep.join((word, str(vocab.wti[word]))) for word in vocab.wti]))
    print("Vocabulary written to " + output_file)


# writes in the following format:
# ndcg for each query
def writeNDCG(nDCGs,file):
    with open(file, 'w') as f:
            f.write("\n".join([str(nDCG) for nDCG in nDCGs]))
    print("NDCGS written to " +file)

def readVocabulary(filename, sep=' '):
    vocab = Vocabulary()
    l = fileLen(filename)
    vocab.curr_ind=l
    vocab.itw = ["" for x in range(l)]
    with open(filename) as f:
        for word in itertools.islice(f, 0, l):
            splt = word.split(sep)
            vocab.wti[splt[0]] = int(splt[1])
            vocab.itw[int(splt[1])] = splt[0]
    return vocab

# returns a list of relevances
def map_to_relevance(qid, dids, qdr):
    relCol = []
    for did in dids:
        rel = qdr[qid][did] if did in qdr[qid] else 0
        relCol.append(rel)
    return relCol

def readNDCG(file):
    ndcg=[]
    with open(file) as f:
        for r in f:
           ndcg.append(float(r.strip()))
    return ndcg


def fileLen(file):
    with open(file) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def plotNDCG(x, y, title):
    fig = plt.figure()
    plt.plot(x, y, color='r')
    plt.xlabel('param')
    plt.ylabel('mNDCG')
    plt.grid(True)
    fig.suptitle(title, fontsize=20)
    plt.savefig(title + '.jpg')
    # plt.show()

def plotNDCG2(old,new,labels,x_labels,title):

    ind = np.arange(len(old))
    width = 0.2
    fig, ax = plt.subplots()
    rec1=ax.bar(ind, old, width, color='r', label=labels[0],align='center')
    rec2=ax.bar(ind+width, new, width, color='g', label=labels[1],align='center')

    ax.set_xticks(ind)
    plt.ylabel("mNDCG")
    ax.set_xticklabels(x_labels)
    plt.grid(True)
    plt.legend()

    plt.savefig(title + '.jpg')

def plotNDCG3(bars,x_labels,title):

    ind = np.arange(len(bars))
    width = 0.2
    fig, ax = plt.subplots()
    rec1=ax.bar(ind, bars, width, color='r',align='center')

    ax.set_xticks(ind)
    plt.ylabel("mNDCG")
    ax.set_xticklabels(x_labels)
    plt.grid(True)
    plt.legend()

    plt.savefig(title + '.jpg')

class Preprocessor(object):
    vocab = None

    def __init__(self, vocab, stemmer=PorterStemmer()):
        self.vocab = vocab
        self.stemmer = stemmer

    def __stem_tokens(self, tokens):
        stemmed = []
        for item in tokens:
            stemmed.append(self.stemmer.stem(item))
        return stemmed

    # type: 0 - documents
    # 1 - queries
    def prep(self, text, type=0):
        lowers = text.lower()
        letters_only = re.sub("[^a-zA-Z]", " ", lowers)
        tokens = nltk.word_tokenize(letters_only)
        filtered = [w for w in tokens if not w in stopwords.words("english")]
        stemmed = self.__stem_tokens(filtered)
        indexed = []
        if stemmed:
            for stem in stemmed:
                # if case if we preprocess query WE DO NOT add new words to vocab.
                if (type == 0):
                    self.vocab.add(stem)
                    indexed.append(self.vocab.wti[stem])
                else:
                    if (stem not in self.vocab.wti):
                        indexed.append(self.vocab.wti['UNKNOWN'])
                    else:
                        indexed.append(self.vocab.wti[stem])
        return indexed


class IndexNode(object):
    def __init__(self, docno, count):
        self.docno = docno
        self.count = count


class InvertedIndex(object):
    TDF = None  # columns: terms, rows:docs/query
    DTF = None  # same but now columns are docs/queries, and rows are terms
    C = 0  # total number of words in the collection

    # m: the vocabulary length
    # type: 0 - doc
    # 1 - query
    def __init__(self, collection, m, type=0):
        # initializing array
        self.TDF = [{} for j in range(m)]
        self.DTF = {}
        for id in collection:
            document = collection[id]
            text = document['fulltext'] if type == 0 else document
            counts = collections.Counter(text)
            for term in counts:
                self.TDF[term][id] = counts[term]
                if (id not in self.DTF):
                    self.DTF[id] = np.zeros(m)
                self.DTF[id][term] = counts[term]
                # self.index[term].append(IndexNode(id, counts[term]))
                self.C += counts[term]

    # did: document id
    def tf(self, terms, did):
        n = len(terms)
        results = np.zeros(n)
        for i in range(n):
            fr = self.TDF[terms[i]][did] if did in self.TDF[terms[i]] else 0  # unseen words in a document get 0 frequency
            results[i] = fr
        return results


    def df(self, terms):
        results = {}
        for word in terms:
            results[word] = len(self.index[word])
        return results

    # collection term frequency
    def ctf(self, terms):
        n = len(terms)
        results = np.zeros(n)
        for i in range(n):
            results[i] = np.sum(self.TDF[terms[i]].values())
        return results


class QLMSmoothing(Enum):
    none = 1
    JR = 2  # Jelinek-Mercer
    DP = 3  # Direchlet Prior
    AD = 4  # Absolute discounting
class PLMkernal(Enum):
    gaussian=1
    triangle=2
    cosine=3
    circle=4
    passage=5


class Vocabulary:
    curr_ind = 0
    itw = None  # index to word
    wti = None  # word to index

    def __init__(self):
        self.curr_ind = 0
        self.itw = []
        self.wti = {}

    def add(self, word):
        if (word not in self.wti):
            self.wti[word] = self.curr_ind
            self.itw.append(word)
            self.curr_ind += 1

# general class for caching
class Cache:
    col=None
    def __init__(self):
        self.col=None
    # r could be an empty array or hash object
    def reset(self):
        self.col=None


<h1>Step 1</h1>

In [ ]:
class NDCG():
    __cache = {}
    __max_rel = None

    def __init__(self, max_rel):
        self.__max_rel = max_rel

    # dcs: documents collections returned by the queries
    # assuming that each document is represented by it's relevance!
    # e.g. [1,4,3,2] <- collection of 4 documents
    def run(self, dc):
        k = len(dc)
        Z = self.__getNorm(k)
        res = 0
        for r in range(k):
            res += self.__score(dc[r], r + 1)
        return Z * res

    # computes ndcg on multiple collections of documents
    # returns an array of values
    def runOnCol(self, dcs):
        n = len(dcs)
        res = np.zeros(n)
        for i in range(n):
            res[i] = self.run(dcs[i])
        return res

    def __getNorm(self, k):
        if (k in self.__cache):
            return self.__cache[k]
        Z = 0
        for r in range(1, k + 1):
            Z += self.__score(self.__max_rel, r)
        Z = 1 / Z
        # storing in cache
        self.__cache[k] = Z
        return Z

    # don't forget that rank starts from 1, and not 0!
    def __score(self, rel, rank):
        return (math.pow(2, rel) - 1) / math.log(1 + rank, 2)

<h1>Part 2</h1>

In [ ]:

class SignTest:
    L = None  # number of loops

    def __init__(self, loops):
        self.L = loops

    # A and B: are the queries with NDCGS
    # returns true or false (two systems are significantly different or not)
    # and p value
    def run(self, A, B):
        n = len(A)
        actualMndcg = np.sum(A)/n - np.sum(B)/n

        testMndcgs = []
        if (n != len(B)):
            raise ValueError('lengths of two systems has to be the same')
        for l in range(self.L):
            # swapping values from two arrays
            for i in range(n):
                pair=self.__select(A[i],B[i])
                A[i] = pair[0]
                B[i] = pair[1]
            testMndcgs.append(np.sum(A)/n-np.sum(B)/n)
        testMndcgs=np.array(testMndcgs)
        p=(np.sum(testMndcgs>actualMndcg)+np.sum(testMndcgs<actualMndcg))/self.L
        return (np.abs(actualMndcg)>np.abs(np.percentile(testMndcgs,95)),p)


    # randomly selects a or b and returns either one
    def __select(self, a, b):
        return np.random.permutation([a, b])

In [ ]:
def test2():
    sTest = SignTest(1000)
    # test from slides
    A=np.array([0.25,0.43,0.39,0.75,0.43,0.15,0.2,0.52,0.49,0.5])
    B=np.array([0.35,0.84,0.15,0.75,0.68,0.85,0.8,0.5,0.58,0.75])
    res = sTest.run(A, B)
    print(res)

    A=np.array([0.25,0.43,0.39,0.75,0.43,0.15,0.2,0.52,0.49,0.5])
    B=np.array([0.25,0.43,0.39,0.75,0.43,0.15,0.2,0.52,0.49,0.5])
    print(str(sTest.run(A, B)[0] == False))

<h1>Step 3</h1>

In [ ]:
# for implementation see the above support section

def run3():

    start=timer()
    file = 'data/collection-small.txt'
    docs, vocab = getDocuments(file)

    print("- Took %.2f sec to load and preprocess docs " % (timer() - start))
    writeVocabulary(vocab,"data/vocab.txt")
    writeDocuments(docs,"data/prep-docs.txt")


    start = timer()
    docs=getDocuments('data/prep-docs.txt',True)
    vocab=readVocabulary('data/vocab.txt')
    print("- Took %.2f sec to load preprocessed docs " % (timer() - start))


    print(docs)

<h1>Step 4</h1>

In [8]:
# for implementation see the above support section
def test4():
    file = 'data/collection-very-small-fake.txt'
    docs, vocab = getDocuments(file)
    ii = InvertedIndex(docs, len(vocab.itw))

<h1>Step 5</h1>

In [ ]:
# query language model
class QLM():
    smoothing = None
    docs = None
    II = None  # inverted index
    params = None  # the set of parameters

    def __init__(self, docs, II, params, smoothing=QLMSmoothing.none):
        self.smoothing = smoothing
        self.II = II
        self.docs = docs
        self.params = params


    # returns k top likely documents for a query
    def findTop(self, qterms, k):
        res = {}
        for did in self.docs:
            res[did] = self.__uniProb(qterms, did)
        sorted_res = sorted(res.iteritems(), key=lambda (k, v): (-v, k))[:k]
        final = []
        for i in range(k):
            final.append(sorted_res[i][0])
        return final


    # returns a unigram probab for the terms given some document model
    # the method is polymorphic (in some sense)
    # it returns different unigram probabilities for different smoothing methods
    def __uniProb(self, terms, did):
        d = len(self.docs[did]['fulltext'])  # number of terms in the document
        if (self.smoothing == QLMSmoothing.none):
            tfs = self.II.tf(terms, did)
            res = (tfs / d)

        if (self.smoothing == QLMSmoothing.JR):
            C = self.II.C
            ctfs = self.II.ctf(terms)
            tfs = self.II.tf(terms, did)
            res = self.params[0] * (tfs / d) + (1 - self.params[0]) * (ctfs / C)

        if (self.smoothing == QLMSmoothing.DP):
            C = self.II.C
            ctfs = self.II.ctf(terms)
            tfs = self.II.tf(terms, did)
            probs = (ctfs / C)
            res = (tfs + self.params[0] * probs) / (d + self.params[0])

        if (self.smoothing == QLMSmoothing.AD):
            tfs = self.II.tf(terms, did)
            C = self.II.C
            ctfs = self.II.ctf(terms)
            du = len(np.unique(self.docs[did]['fulltext']))  # unique words in the document
            probs = (ctfs / C)
            res = ((np.maximum.reduce([tfs - self.params[0], np.zeros(len(terms))])) / d)\
                  + self.params[0] * du * probs / d
        return np.sum(np.log(res))

    # used only for crash tests
    def testll(self, did):
        fac = scipy.misc.factorial
        terms = self.docs[did]['fulltext']
        un = np.unique(terms)
        L = len(terms)
        tfs = self.II.tf(terms, did)
        utfs = self.II.tf(un, did)
        K = (fac(L) / np.prod(fac(utfs)))

        return K * ((np.prod(tfs / L)))

def test5():
    file = 'data/collection-very-small.txt'
    docs, vocab = getDocuments(file)
    dii = InvertedIndex(docs, len(vocab.itw))
    file = 'data/queries-small.txt'
    queries = getQueries(file, vocab)

    file = 'data/qrels-small.txt'
    qdr = getDocQuerRel(file)

    qlm = QLM(docs, dii, params=[0.5], smoothing=QLMSmoothing.AD)
    res = {}
    ndcg = NDCG(2)
    for qid in queries:
        dids = qlm.findTop(queries[qid], 10)
        # map to relevance
        relDocs = map_to_relevance(qid, dids, qdr)
        res[qid] = {'docs': relDocs, 'ndcg': ndcg.run(relDocs)}
    print(res)

<h1>Greed search over parameters space</h1>
<h2>Jelinek-Mercer</h2>
<img src="plots/jelinek_mercer.jpg" style="width:400px"></img>
<h2>Direchlet prior</h2>
<img src="plots/dirichlet_prior.jpg" style="width:400px"></img>
<h2>Absolute discounting</h2>
<img src="plots/absolute_discounting.jpg" style="width:400px"></img>


<h1>Model comparison</h1>
<table>
<tr><th>Model</th><th>Optimal parameter</th><th>mNDCG</th></tr>
<tr>
<td>Jelinek-Mercer</td><td>0.2</td><td>0.164</td>
</tr>
<tr>
<td>Direchlet prior</td><td>1000</td><td>0.281</td>
</tr>
<tr>
<td>Absolute discounting</td><td>0.9</td><td>0.16</td>
</tr>
</table>

<h1>Step 6</h1>
In this part the author has been using sampling in several places in the optimization to speed-up computations. The method appears to produce overflows and underflows and therefore that has been taking into account during the implementation phase.


In [ ]:

class TwoStageSmoothing():
    params = None  # first one is miu(for Dirichet) and the second one is lambda (for JM)
    pi=[]
    docs = None  # the collection of documents
    queries=None # the collection of queries
    II = None  # inverted index
    iter=None
    perc=None
    cache=None
    debug=None

    # iter: number of iterations in the training phases
    # perc: percent of documents that are sampled each time e.g. 0.1
    def __init__(self, docs,queries, II, params,perc, iter=100,debug=False):
        self.II = II
        self.docs = docs
        self.queries=queries
        self.params = params
        self.perc=perc
        self.iter=iter
        self.pi=np.ones(len(docs))/len(docs)
        self.debug=debug
        self.cache=Cache() # we shall cache denom. of E-step


    def findTop(self, qterms, k):
        res = {}
        for did in self.docs:
            res[did] = self.__uniProb(qterms, did)
        sorted_res = sorted(res.iteritems(), key=lambda (k, v): (-v, k))[:k]
        final = []
        for i in range(k):
            final.append(sorted_res[i][0])
        return final

    def __uniProb(self, terms, did):
        # estimating collection based normalization of non-discr. terms
        # a.k.a Jelinek-Mercel
        cProbs = self.jm(terms)

        # estimating document prob. (Dirichlet)
        dProbs=self.dir(terms,did)

        res=(1-self.params[1])*dProbs +self.params[1]*cProbs

        return np.sum(np.log(res))

    # log likelihood for documents model
    def dLL(self):
        res=0
        for did in self.docs:
            terms=self.docs[did]['fulltext']
            d=len(terms)
            tfs = self.II.tf(terms, did)
            cProbs = self.jm(terms)
            dProbs= (tfs -1 + self.params[0] * cProbs) / (d -1 + self.params[0])
            res+=np.sum(np.log(dProbs))
        return res

    # log likelihood for queries model
    def qLL(self):
        res=1
        for qid in self.queries:
            temp_res=0
            for i,did in enumerate(self.docs):
                pi=self.pi[i]
                terms=self.queries[qid]
                dProd=self.dir(terms,did) # 1st role prob, prob of query qid
                cProb = self.jm(terms) # the JM 2nd role prob
                temp_res+=pi*np.prod((1-self.params[1])*dProd + self.params[1]*cProb)
            res*=temp_res
        return res


    # p(ids| d_i) #
    # returns a list of dirichlet smoothed probabilities for term ids
    def dir(self,terms,did):
        d=len(self.docs[did]['fulltext'])
        tfs = self.II.tf(terms, did)
        cProbs = self.jm(terms)
        res=(tfs+ self.params[0] * cProbs) / (d + self.params[0])
        if(np.array(res<0).any() or np.array(res>1).any()):
            print('error in dir')
        return res

    # return JM list of probabilities
    # note that JM play the second role, similar to TF-IDF
    def jm(self,terms):
        C = self.II.C
        return self.II.ctf(terms)/C

    def train(self,llEveryIter):
        print('phase 1 of training')
        self.__trainStage1(llEveryIter)
        print('phase 2 of training')
        self.__trainStage2(llEveryIter)


    def __trainStage1(self,n):
        col=[]
        par=[]
        for i in range(1,self.iter+1):
            deriv=self.__dGrads()
            self.params[0]-=deriv[0]/deriv[1]
            if(self.debug==True and i%n==0):
                LL=self.dLL()
                col.append(LL)
                par.append(self.params[0])
                print('iter # '+str(i))
                print('dLL is : '+str(LL))
        self.__plotProgress(col,par,'Phase 1 (miu1) dLL')

    def __trainStage2(self,n):
        col=[]
        par=[]
        for j in range(1,self.iter+1):
            # E-step
            for qid in random.sample(self.queries,20):
                new_pi=[]
                self.cache.reset() # resetting cache
                for i in range(len(self.pi)):
                    new_pi.append(self.__E(i,qid))
                self.pi=new_pi
            for qid in random.sample(self.queries,5):
            # M-step
                self.params[1]=self.__M(qid)
            if(j%n==0):
                LL=self.qLL()
                col.append(LL)
                par.append(self.params[1])
                print('iter # '+str(j))
                print('qLL is : '+str(LL))
        self.__plotProgress(col,par,'Phase 2 (lambda1) qLL')



    # returns a tuple of 1st order derivative and 2nd order derivative
    def __dGrads(self):
        grad1=0
        grad2=0
        samp = random.sample(self.docs, int(math.floor(self.perc*len(self.docs))))

        for did in samp:
            terms=self.docs[did]['fulltext']
            d=len(terms)
            tfs = self.II.tf(terms, did)
            cProbs = self.jm(terms)

            # temp collector for 1st order der
            tRes1= (tfs*((d-1)*cProbs-tfs+1))/\
                 ((d-1+self.params[0])*(tfs-1 + self.params[0]*cProbs))

            # temp collector of 2nd order der
            tRes2=(tfs*np.power(((d-1)*cProbs-tfs+1),2))/\
                 math.pow((d-1+self.params[0]),2)*np.power(tfs-1+self.params[0]*cProbs,2)

            grad1+=np.sum(tRes1)
            grad2-=np.sum(tRes2)

        return (grad1,grad2)

    def __E(self,i,qid):

        # computing numerator
        did=self.docs.keys()[i]
        terms=self.queries[qid]
        try:
            num=self.pi[i]*np.prod((1-self.params[1])*self.dir(terms,did)+self.params[1]*self.jm(terms))
        except ValueError:
            print('value error in E')
        # computing denominator
        if(self.cache.col==None):
            denom=0
                # over all docs
            for k,did in enumerate(self.docs):
                denom+=self.pi[k]*np.prod((1-self.params[1])*self.dir(terms,did)+self.params[1]*self.jm(terms))
            self.cache.col=denom
        else:
             denom=self.cache.col

        if(denom==0 or num==0):
            res=0.0001 # if we set it to 0 there is no way back.
        else:
            res=num/denom

        # this twoStage model SUFFERS FROM OVERFLOWS!
        if(math.isnan(res) or math.isinf(res)):
            res=0.0001  # if we set it to 0 there is no way back.
           # print('error in E-step')
        return res


    def __M(self,qid):
        Q=len(self.queries)
        res=0
        for i,did in enumerate(self.docs):
            terms=self.queries[qid]
            num = self.params[1]*self.jm(terms)
            denom= (1-self.params[1])*self.dir(terms,did)+self.params[1]*self.jm(terms)
            # avoid zeroes
            ind=np.where(denom==0)
            if len(ind[0])!=0:
                for i in ind[0]:
                    num[i]=0
                    denom[i]=1
            res+=self.pi[i]*np.sum(num/denom)
            if(res/Q==0 or math.isnan(res/Q)):
                print('error in M step')
        return res/Q

    def __plotProgress(self,col,iter,title):
        fig = plt.figure()
        plt.plot(iter,col, color='r')
        plt.xlabel('param')
        plt.ylabel('mNDCG')
        plt.grid(True)
        fig.suptitle(title, fontsize=20)
        plt.savefig(title.replace(' ','_').lower())

In [ ]:
def run6():
    docs=getDocuments('data/prep-docs.txt',True,20000)
    vocab=readVocabulary('data/vocab.txt')
    dii = InvertedIndex(docs, len(vocab.itw))
    file = 'data/queries-small.txt'
    queries = getQueries(file, vocab)

    file = 'data/qrels-small.txt'
    qdr = getDocQuerRel(file)

    twoStage=TwoStageSmoothing(docs,queries,dii,[1,0.5],perc=0.1,iter=2,debug=False)
    # train two parameters
    twoStage.train(5)
    ndcg=NDCG(2)
    ndcgRes = []
    Q=len(queries)
    for qid in queries:
        dids = twoStage.findTop(queries[qid], 10)
        # map to relevance
        relDocs = map_to_relevance(qid, dids, qdr)
        ndcgRes.append(ndcg.run(relDocs))
    writeNDCG(ndcgRes,"ndcg/twoStage.txt")
    print(str(np.sum(twoStage.pi)))
    print('with parameters: '+str(twoStage.params)+' mndcg is: '+str(np.sum(ndcgRes)/Q))

    # optimized params: \mu =1.0007869864702941, \lambda=1.6292281580652353e-10]

<h1>Conclusions</h1>

The model is much slower than simple QLM with smoothing, therefore it was not realistic to run the model on full dataset, and only 20,000 documents have been used with sampling during the training phase. It has been noticed that the model is sensitive to the number of documents it is trained on and therefore the produced result most likely is sub-optimal. In addition, multiple iterations in the training phase(epoches) has been notice to produce positive effect to the final evaluation measure (mNDCG).


<h2>Optimized Parameters</h2>
\mu = 1.0007869864702941<br>
\lambda =\lambda=1.6292281580652353e-10

<h2>mNDCG</h2>
mNDCG=0.209192471774<br>
But as has been noticed, it is possible to improve the result by running it on more data. And as models from step 3 have been trained on more data, we can't objectively compare the current and previous models. 

<h1>Step 7 (Bonus)<h1>

In [ ]:
# Position Language Model
class PLM():
    docs=None
    queries=None
    kernal=None
    dii=None # document inverted index
    sigma=None

    def __init__(self,docs,queries,dii,params,kernal=PLMkernal.gaussian):
        self.docs=docs
        self.queries=queries
        self.kernal=kernal
        self.dii=dii
        self.sigma=params

    def __score(self,qid,did,i):
        n=len(qid)
        plms=self.__plmProb(self.queries[qid],did,i)
        return -np.sum((1/n)*np.log((1/n)/plms))

    # the special tf
    # i: position
    # returns a list of tfs for the terms
    def __tf(self,terms,i):
        res=np.zeros(len(terms))
        for j,did in enumerate(self.docs):
            tfs=self.dii.tf(terms,did)
            k=self.__k(i,j)
            res+=tfs*k
        return res

    # the actual position language model
    # terms: a list of terms
    # i: position
    def __plmProb(self,terms,did,i):
        tfs = self.__tf(terms,i)
        # computing denominator
        # in optimized way
        Z = 0
        for j in range(len(self.docs)):
            Z+=self.__k(i,j)
        return tfs/Z

    def reEval(self,qid,dids):
            score={}
            for did in dids:
                score[did]=np.max([self.__score(qid,did,i) for i in range(len(dids))])
            res=sorted(score.items(), key=operator.itemgetter(1),reverse=True)
            res= [r[0] for r in res]
            return res

    # polynomic kernal function
    def __k(self,i,j):

        # 1. gaussian kernal
        if(self.kernal==PLMkernal.gaussian):
            return math.exp(-math.pow(i-j,2)/2*math.pow(self.sigma,2))

        # 2. triangle kernal
        if(self.kernal==PLMkernal.triangle):
            if(np.abs(i-j)<=self.sigma):
                return 1- np.abs(i-j)/self.sigma
            else:
                return 0

        # 3. cosine (hamming ) kernal
        if(self.kernal==PLMkernal.cosine):
            if(np.abs(i-j)<=self.sigma):
                return (1+math.cos((np.abs(i-j)*math.pi)/self.sigma))/2
            else:
                return 0

        # 4. circle kernal
        if(self.kernal==PLMkernal.circle):
             if(np.abs(i-j)<=self.sigma):
                 return math.sqrt(1-math.pow((np.abs(i-j)/self.sigma),2))
             else:
                return 0

        # 5. passage
        if(self.kernal==PLMkernal.passage):
            if(np.abs(i-j)<=self.sigma):
                return 1
            else:
                return 0

<h1>Results</h1>
<img src="plots/PLM_and_direchlet_LM_eval_(mNDCG).jpg" style="width:400px"></img>
<h1>Conclusions</h1>
As has been noticed during the experiments the kernals do not different from each other in the rescored documents mNDCG. The reason for this might be that we have only let the model to rescore the obtained documents via Dirichlet smoothed QLM, but it could have different effect if it would retrieve top 10 matches itself.

Computational-wise the model is more expensive than simple Dirichlet smoothed QLM and as we can see from the results, the latter is more accurate. Once again, the results might be skewed as we did only re-scoring. 

<h1>Step 8</h1>

In [ ]:
class MySentences(object):
    docs=None
    vocab=None
    def __init__(self, docs,vocab):
         self.docs=docs
         self.vocab=vocab

    def __iter__(self):
            for did in self.docs:
                yield [self.vocab.itw[int(x)] for x in self.docs[did]['fulltext']]

def run8():
    docs=getDocuments('data/prep-docs.txt',True)
    vocab=readVocabulary('data/vocab.txt')
    sentences = MySentences(docs,vocab) # a memory-friendly iterator
    model = gensim.models.Word2Vec(sentences,min_count=1,workers=5,window=10,size=200,negative=15,sg=1)
    print('evaluation')
    model.accuracy('eval/questions-words.txt')

<h1>Conclusions</h1>

The model has been tried with different parameters, and tested on questions test-set. It appeared that skip-gram outperforms CBOW and 200 is a good choise for the number of hidden units. In addition, negative sampling has been used to speed-up computations. In constrast to hierarchical softmax, negative sampling makes it possible to use output represention of words, while the former makes the output matrix correspond to inner node weights on the Huffman's tree. The number of negative samples has been noticed affect positivly the evaluation measure, and the number has been set to 15 to train more accurate model. The context window size has been set to 10, similar to the original Mikolov's paper and the minimum number of words set to 1, while the default was 5. 

<h1>Step 9</h1>
As it was permitted in the homework assignment to use doc2vec(paragraph2vec), the author has used ginsem implementation of the model. 

In [ ]:
# evaluates the document ranks for queries
class VecEvaluator():
    docs=None # documents with vectors
    queries = None # queries with vectors

    def __init__(self,docs,queries):
        self.docs=docs
        self.queries=queries

    # re-evaluates based on cosine similarity
    def eval(self,qid,dids):
        cos={}
        for did in dids:
            cos[did]=1-spatial.distance.cosine(self.docs[did]['vec'], self.queries[qid]['vec'])
        res=sorted(cos.items(), key=operator.itemgetter(1),reverse=True)
        #res=sorted(cos,key=cos.get, reverse=True)
        res= [r[0] for r in res]
        return res

class LabeledLineSentence(object):
    col=None # collection of documents or queries
    vocab=None
    type=None
    # type: 0 for documents, 1 for queries
    def __init__(self, col,vocab,type=0):
         self.col=col
         self.vocab=vocab
         self.type=type
    def __iter__(self):
        for i,id in enumerate(self.col):
            if(self.type==0):
                yield gensim.models.doc2vec.LabeledSentence(words=[self.vocab.itw[int(x)] for x in self.col[id]['fulltext']],tags=['SENT_%s' % i])
            if(self.type==1):
                words=[self.vocab.itw[int(x)] for x in self.col[id]]
                yield gensim.models.doc2vec.LabeledSentence(words=words,tags=['SENT_%s' % i])

In [ ]:

def run9():
    docs=getDocuments('data/prep-docs.txt',True)
    vocab=readVocabulary('data/vocab.txt')
    queries = getQueries('data/queries-small.txt', vocab)
    dSent = LabeledLineSentence(docs,vocab) # a memory-friendly iterator
    qSent = LabeledLineSentence(queries,vocab,1) # a memory-friendly iterator

    print('starting training')
    dModel = gensim.models.Doc2Vec(dSent,workers=7,window=10,size=100,alpha=0.1,negative=15,iter=1)
    qModel = gensim.models.Doc2Vec(qSent,workers=5,window=10,size=100,alpha=0.1,negative=15,iter=1)

    print ('finished training')
    # storing vector representations
    for i,did in enumerate(docs):
        docs[did]['vec']=dModel.docvecs['SENT_%s' % i]
    for i,qid in enumerate(queries):
        terms=queries[qid]
        queries[qid]={'fulltext':terms,'vec':qModel.docvecs['SENT_%s' % i]}

    file = 'data/qrels-small.txt'
    qdr = getDocQuerRel(file)
    ev= VecEvaluator(docs,queries) # evaluator

    ndcgRes=[]
    ndcg = NDCG(2)
    Q=len(queries)
    # taking top 10 documents for each query based on similarity
    for qid in queries:
        dids=  ev.eval(qid,docs.keys())

         # map to relevance
        relDocs1 = map_to_relevance(qid, dids, qdr)
        ndcgRes.append(ndcg.run(relDocs1))


    writeNDCG(ndcgRes,"ndcg/vectorized.txt")
    print("mNDCG is: ")
    print(str(np.sum(ndcgRes) / Q))

<h1>Conclusions</h1>
The model shares similarities with the previous one, and therefore some settings such as
negative sampling with 15 n.s. have been used. 
The number of hidden units has been set to 100 as a trade-off between quality of vectors and the time-complexity. The distributed bag of words from the paper produced better result and the same has been supported by the author's own results, it has been used in later phases. In addition, the author tried different learning rate values, and 0.1 appears to be a good choise. 

<h1>Step 10 </h1>

In [ ]:
def run10():
    docs=getDocuments('data/prep-docs.txt',True)
    vocab=readVocabulary('data/vocab.txt')
    dii = InvertedIndex(docs, len(vocab.itw))

    queries = getQueries('data/queries-small.txt', vocab)
    dSent = LabeledLineSentence(docs,vocab) # a memory-friendly iterator
    qSent = LabeledLineSentence(queries,vocab,1) # a memory-friendly iterator

    print('training dModel')
    dModel = gensim.models.Doc2Vec(dSent,workers=6,min_count=1,window=10,size=200,alpha=0.1,negative=15,iter=3)

    print('training qModel')
    qModel = gensim.models.Doc2Vec(qSent,workers=6,min_count=1,window=10,size=200,alpha=0.1,negative=15,iter=3)

    print ('finished training')
    # storing vector representations
    for i,did in enumerate(docs):
        docs[did]['vec']=dModel.docvecs['SENT_%s' % i]
    for i,qid in enumerate(queries):
        terms=queries[qid]
        queries[qid]={'fulltext':terms,'vec':qModel.docvecs['SENT_%s' % i]}

    print('finished extracting vectors')

    file = 'data/qrels-small.txt'
    qdr = getDocQuerRel(file)

    ## optimals are: JR - 0.2, Dir: 1000, AD: 0.9

    sms = [QLMSmoothing.JR, QLMSmoothing.DP, QLMSmoothing.AD]
    smsTitles = ['Jelinek-Mercer', 'Dirichlet_prior', 'Absolute_discounting']
    params = [0.2, 1000, 0.9]
    ndcg = NDCG(2)
    re= VecEvaluator(docs,queries) # re-evaluator
    oldCol = np.zeros(len(smsTitles))
    newCol = np.zeros(len(smsTitles))
    Q = len(queries)

    for i in range(len(sms)):
        sm = sms[i]
        param=params[i]
        qlm = QLM(docs, dii, params=[param], smoothing=sm)
        ndcg1Res = []
        ndcg2Res = []
        for qid in queries:
            dids1 = qlm.findTop(queries[qid]['fulltext'], 10) # original dids
            dids2=  re.eval(qid,dids1)

            # map to relevance
            relDocs1 = map_to_relevance(qid, dids1, qdr)
            relDocs2 = map_to_relevance(qid, dids2, qdr)

            ndcg1Res.append(ndcg.run(relDocs1))
            ndcg2Res.append(ndcg.run(relDocs2))

        writeNDCG(ndcg1Res,"ndcg/"+smsTitles[i].lower()+".txt")
        writeNDCG(ndcg2Res,"ndcg/"+smsTitles[i].lower()+"_vectorized.txt")
        oldCol[i]=np.sum(ndcg1Res) / Q
        newCol[i]=np.sum(ndcg2Res) / Q

    print(newCol)
    plotNDCG2(oldCol,newCol,x_labels=smsTitles,labels=["QLM","Vector-based re-eval"],title="QLM and Vector-based eval (mNDCG)")

run10()

<h1>Results</h1>
<img src="plots/QLM_and_vector_based_eval(mNDCG).jpg" style="width:400px"></img>

<h1>Conclusions</h1>
As has been observed, vector-based re-evaluation produces worse mNDCG. But as previously, the model has been used only for re-ranking, and therfore if used on the full set of documents instead of top 10 retrieved by smootheded QLMs it could have produced better results.
Nevertheless, we can see that it has very close to the original mNDCG and it has been observed that more iterations produce better results, and therefore it has been set to 3 after several tests as a trade-off between quality of vectors and computation time.




<h1>Step 11</h1>

As the base-line for comparisons the smoothed Dirichlet QLM has been chosen. 
<table>
<tr>
<th>Model</th><th>mNDCG</th><th>p-value</th><th>sign. different?</th></tr>
<tr>
<td>Dirichlet</td><td><b>0.284</b></td><td>-</td><td>-</td>
</tr>
<tr>
<td>Absolute discounting</td><td>0.164</td><td>1</td><td>True</td>
</tr>
<tr>
<td>Jelinek-Mercer</td><td>0.116</td><td>1</td><td>True</td>
</tr>
<tr>
<td>Dirichlet vectorized (re-rank)</td><td>0.264</td><td>1</td><td>True</td>
</tr>
<tr>
<td>Absolute discounting vectorized (re-rank)</td><td>0.134</td><td>1</td><td>True</td>
</tr>
<tr>
<td>Jelinek-Mercer vectorized (re-rank)</td><td>0.102</td><td>1</td><td>True</td>
</tr>
<tr>
<td>TwoStage smoothing</td><td>0.21</td><td>1</td><td>True</td>
</tr>
<tr>
<td>PLM circle</td><td>0.25</td><td>1</td><td>True</td>
</tr>
<tr>
<td>PLM cosine</td><td>0.25</td><td>1</td><td>True</td>
</tr>
<tr>
<td>PLM gaussian</td><td>0.25</td><td>0.99</td><td>False</td>
</tr>
<tr>
<td>PLM passage</td><td>0.25</td><td>0.97</td><td>False</td>
</tr>
<tr>
<td>PLM triangle</td><td>0.25</td><td>0.75</td><td>False</td>
</tr>

</table>

<h1>Conclusions</h1>

1. Dirichlet smoothed QLM showed the best mNDCG results but required to perform
greed search over a space, which could be inefficient for large documents and quaries collection.
An alternative could be to learn the \mu parameter using machine learning techniques
2. Vectorized re-ranking of 3 smoothed methods showed worse results than the originals. Unfortunately, when the vector-based method has been used to do a pure top 10 documents ranking for each query, it has produced close to 1% mNDCG results. The problem could reside in the fact that word2vec techniques are less efficient for IR tasks, the same for entailment problems. It is possible that the word2vec model has to be trained with more IR problem oriented objective function. 
3. Position Language models showed also worse re-ranking results, but hypothesis testing indicated that 3/4 models are not significantly different. Unfortunately, it has not been possible to perform a pure top 10 documents ranking for each query to see how well the models perform.
4. TwoStage smoothing is a more complex method than the 3rd step smoothing methods, but it encodes both Dirichlet prior and Jelinek-Mercel smoothing,as the former is better for unknown words, and the latter for balancing frequent words in the collection. Unfortunately, the model requires two stage training which is time consuming and therefore it was not possible to use the full collection of documents to perform ranking. Nevertheless, it has been noticed that model improves performences significantly when 10,000 documents are changed to 20,000. Thus, it could a good candidate for futher research. 
